#  Load libraries

In [32]:
import torch
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np
from ipywidgets import interact, interactive, FloatSlider, IntSlider
import ipywidgets as widgets
torch.manual_seed(123)

# Dynamical Systems

### Notation

Notation for functions $f,g,h$ and vectors $x,y,z$ we use $\circ$ to denote function composition as well as application of a vector function

$$
\begin{bmatrix}
f \\
g \\
h \\
\end{bmatrix}
\circ
\left(
\begin{bmatrix}
x \\
y \\
z \\
\end{bmatrix}
\right)
=
\begin{bmatrix}
f(x) \\
g(y) \\
h(z) \\
\end{bmatrix}
$$

and 

$$
\begin{bmatrix}
a \\
b \\
c \\
\end{bmatrix}
\circ
\begin{bmatrix}
f \\
g \\
h \\
\end{bmatrix}
\circ
\left(
\begin{bmatrix}
x \\
y \\
z \\
\end{bmatrix}
\right)
=
\begin{bmatrix}
a(f(x)) \\
b(g(y)) \\
c(h(z)) \\
\end{bmatrix}
$$

and so on.



We use $\odot$ to represent the Hadamard product of two matrices $A,B$

$$
A \odot B = 
\begin{bmatrix}
A_{0,0} & A_{0,1} \\
A_{1,0} & A_{1,1} \\
\end{bmatrix}
\odot
\begin{bmatrix}
B_{0,0} & B_{0,1} \\
B_{1,0} & B_{1,1} \\
\end{bmatrix}
=
\begin{bmatrix}
A_{0,0}B_{0,0} & A_{0,1}B_{0,1} \\
A_{1,0}B_{1,0} & A_{1,1}B_{1,1} \\
\end{bmatrix}
$$


$\sigma$ is an arbitrary "activation" function. I.e., $\sigma$ is applied entry-wise to a vector or matrix.

$I$ is the identity activation function such that $I(x)=x$.




### Basic MLP

Consider an MLP N.  For an $l$-layer network this is commonly written as

$$
\sigma(W_{l-1}\; \sigma(W_{l-2}\; \cdots \sigma(W_0\; x  + b_0) \cdots + b_{l-2}) + b_{l-1})
$$

for "weight" matrices $W_i$, "bias" vectors $b_i$, and input data $x$.

### Dynamical systems

A classic discrete or algebraic dynamical system is a *fixed* function that is iterated such as in

$$
f(f(\cdots f(x) \cdots))=f
$$

https://en.wikipedia.org/wiki/Dynamical_system

Famous examples abound!   

The logstic map https://en.wikipedia.org/wiki/Logistic_map

![](https://upload.wikimedia.org/wikipedia/commons/thumb/5/50/Logistic_Bifurcation_map_High_Resolution.png/330px-Logistic_Bifurcation_map_High_Resolution.png)

The Lorenz attractor https://en.wikipedia.org/wiki/Lorenz_system

![](https://upload.wikimedia.org/wikipedia/commons/thumb/5/5b/Lorenz_attractor_yb.svg/330px-Lorenz_attractor_yb.svg.png)

Baker's map https://en.wikipedia.org/wiki/Baker%27s_map

![](https://upload.wikimedia.org/wikipedia/commons/thumb/7/7d/Ising-tartan.png/330px-Ising-tartan.png)

### Building a bridge between dynamical systems and neural networks

As we will demonstrate, dynamical systems and MLPs share the idea of recursive/iterative maps.  For example, setting 

$$
f(x)  = \sigma(W x + b)
$$ 

we get a dynamical system

$$
\sigma(W\; \sigma(W\; \cdots \sigma(W\; x  + b) \cdots + b) + b)
$$

which is a cousin of the MLP

$$
\sigma(W_{l-1}\; \sigma(W_{l-2}\; \cdots \sigma(W_0\; x  + b_0) \cdots + b_{l-2}) + b_{l-1})
$$

with one **important difference**.   For the MLP $W_i$ and $b_i$ *vary* from iteration to iteration, while in the dynamical system, $W$ and $b$ are *fixed* from iteration to iteration.  
In other words, this is exactly a neural network, except we have simplified it by **sharing weights** between all of the layers.


The question we wish to explore is, what really are the differences between the two approaches?

### Simple example

<img src="https://i.pinimg.com/originals/dd/bf/6a/ddbf6a19df36ca75e22fa1d379957666.png" width="500"/>

Let's make a dynamical system and boil it down to its essence.   In particular, we know that NNs can have quite complicated behavior with their maps that change every iteration.  We therefore wish to understand if dynamical systems can also evince a similar level of complexity, even though the are hobbled by only being able to iterate a *fixed* map.

With this in mind, let us consider a very simple dynamical system.  Namely, let us consider the same dynamical system above by iteration the function

$$
f(x)  = \sigma(W x + b)
$$ 

We begin our simplification by letting $x$, $W$, and $b$ be scalar, leading to the 1D map

$$
f(x) = \sigma(w\; x + b)
$$

Futher, let $\sigma$ be a simple non-linearity, such as the square function, leading to

$$
f(x) = (w\;x + b)^2
$$

### A simple numerical experiment

In [33]:
def f(x,w,c):
    return (w*x+c)**2

In [34]:
# Interactive visualization with improved widget styling
def plot_dynamical_system(w=1.0, c=0.0, x=0.0):
    """
    Interactive plot of dynamical system f(x) = (wx + c)^2
    
    Parameters:
    - w: weight parameter
    - c: bias parameter  
    - x: initial condition
    """
    layers = 50
    record_x = np.zeros(layers)
    
    for i in range(layers):
        record_x[i] = x
        x = f(x, w, c)
    
    # Create subplots using Plotly
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Full Trajectory', 'Last 10 Points'),
        horizontal_spacing=0.1
    )
    
    # Plot full trajectory
    fig.add_trace(
        go.Scatter(
            y=record_x,
            mode='lines+markers',
            name='Trajectory',
            line=dict(color='blue', width=2),
            marker=dict(size=4, color='red')
        ),
        row=1, col=1
    )
    
    # Plot last 10 points
    fig.add_trace(
        go.Scatter(
            y=record_x[-10:],
            mode='lines+markers',
            name='Last 10',
            line=dict(color='green', width=2),
            marker=dict(size=6, color='orange')
        ),
        row=1, col=2
    )
    
    # Update layout
    fig.update_layout(
        title=f'Dynamical System: f(x) = ({w:.2f}x + {c:.2f})²',
        showlegend=False,
        height=400,
        width=800
    )
    
    fig.update_xaxes(title_text="Iteration", row=1, col=1)
    fig.update_xaxes(title_text="Iteration", row=1, col=2)
    fig.update_yaxes(title_text="x", row=1, col=1)
    fig.update_yaxes(title_text="x", row=1, col=2)
    
    fig.show()

# Create interactive widget with better styling
w_slider = FloatSlider(
    value=1.0, min=-1.0, max=1.0, step=0.01,
    description='Weight (w):',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='400px')
)

c_slider = FloatSlider(
    value=0.0, min=-2.0, max=2.0, step=0.01,
    description='Bias (c):',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='400px')
)

x_slider = FloatSlider(
    value=0.0, min=-1.0, max=1.0, step=0.01,
    description='Initial x:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='400px')
)

# Create interactive display
interactive_plot = interactive(
    plot_dynamical_system,
    w=w_slider,
    c=c_slider, 
    x=x_slider
)

# Display with better layout
display(interactive_plot)

interactive(children=(FloatSlider(value=1.0, description='Weight (w):', layout=Layout(width='400px'), max=1.0,…

Of course we can see them all at once but just looping over values of $b$.   What do we get?  A cousin of the logistic map!

In [35]:
import numpy as np
import plotly.graph_objects as go
from IPython.display import display

def create_bifurcation_diagram(w=1, layers=40, num_bs=500):
    """
    Create bifurcation diagram showing the behavior of the dynamical system
    as the bias parameter c varies. Uses Plotly's crossfilter.enabled to show
    vertical line at mouse position.
    """
    points = []
    c_values = np.linspace(0, -2, num_bs)
    
    for c in c_values:
        x = 0
        for j in range(layers):
            x = f(x, w, c)
            # Record last 20 points for each c value
            if j > layers - 20:
                points.append([c, x])
    
    points = np.array(points)
    
    # Create figure
    fig = go.Figure()
    
    # Add scatter plot with enhanced hover information
    fig.add_trace(go.Scatter(
        x=points[:, 0],
        y=points[:, 1],
        mode='markers',
        marker=dict(
            size=1.5,
            color=points[:, 1],
            colorscale='Viridis',
            opacity=0.7
        ),
        hovertemplate='<b>c-parameter:</b> %{x:.4f}<br>' +
                     '<b>Value:</b> %{y:.4f}<br>' +
                     '<extra></extra>'
    ))
    
    # Update layout with crossfilter spike for vertical line
    fig.update_layout(
        title={
            'text': 'Bifurcation Diagram: Cousin of the Logistic Map<br>' +
                   '<sub>Hover over points to see c-value details</sub>',
            'x': 0.5,
            'xanchor': 'center'
        },
        xaxis_title='Bias Parameter (c)',
        yaxis_title='Values (x)',
        width=900,
        height=600,
        showlegend=False,
        hovermode='closest',
        xaxis=dict(
            showspikes=True,
            spikecolor="red",
            spikethickness=2,
            spikedash="dash",
            spikemode="across",
            spikesnap="cursor"
        ),
        yaxis=dict(
            showspikes=True,
            spikecolor="red", 
            spikethickness=1,
            spikedash="dot",
            spikemode="across"
        )
    )
    
    # Add annotation box for current c-value
    fig.add_annotation(
        x=0.02, y=0.98,
        xref="paper", yref="paper",
        text="Move mouse over plot to see c-value line",
        showarrow=False,
        bgcolor="rgba(255,255,255,0.8)",
        bordercolor="gray",
        borderwidth=1,
        font=dict(size=10)
    )
    
    return fig

# Create and display the bifurcation diagram
bifurcation_fig = create_bifurcation_diagram()
bifurcation_fig.show()

### A more *complex* example

If I can be forgiven a small pun, what happens if $b$ is allowed to be complex?  In other words, let's look at 
values of $b$ taken from the complex plane, and just ask "does the scalar neural network with shared weights converge
or go off to infinity (when started at $x=0$)"?  I mean, the set of $b$ for which the scalar neural network (remember with shared weights)
gives a finite value as we add more and more layers must be simple, right?

In [38]:
import numpy as np
import plotly.graph_objects as go
from ipywidgets import interact, IntSlider, interactive, widgets
from IPython.display import display

def f(z, w, c):
    return z**2 + c

def create_mandelbrot_visualization(layers=50):
    """
    Create Mandelbrot set visualization using complex dynamical system
    f(z) = z² + c with z₀ = 0
    """
    # Create complex grid
    resolution = 300  # Higher resolution for better quality
    b_real, b_imag = np.meshgrid(
        np.linspace(-3, 1, resolution), 
        np.linspace(-1, 1, resolution)
    )
    c = b_real + 1j * b_imag
    
    # Initialize with zeros (starting point z₀ = 0)
    z = np.zeros_like(c)
    w = 1.0
    
    # Iterate the dynamical system
    for i in range(layers):
        z = f(z, w, c)
        # Clip to prevent overflow
        z = np.clip(z, -5+5j, 5+5j)
    
    # Determine which points are in the set (bounded)
    mandelbrot_set = np.abs(z) < 2.0
    
    # Create the plot
    fig = go.Figure()
    
    fig.add_trace(go.Heatmap(
        z=mandelbrot_set.astype(int),
        x=np.linspace(-3, 1, resolution),
        y=np.linspace(-1, 1, resolution),
        colorscale='Hot',
        showscale=False,
        hoverongaps=False
    ))
    
    fig.update_layout(
        title=f'Mandelbrot Set (Neural Network Perspective) - {layers} Layers',
        xaxis_title='Real(c)',
        yaxis_title='Imag(c)',
        width=600,
        height=400,
        xaxis=dict(scaleanchor="y", scaleratio=1),  # Equal aspect ratio
        yaxis=dict(constrain='domain')
    )
    
    fig.show()

# Interactive Mandelbrot with layer control
def interactive_mandelbrot(layers):
    create_mandelbrot_visualization(layers)

# Create widget for layer control
layers_slider = IntSlider(
    value=2, min=1, max=100, step=1,
    description='Layers:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='400px')
)

# Create interactive display
mandelbrot_interactive = interactive(
    interactive_mandelbrot,
    layers=layers_slider
)

display(mandelbrot_interactive)

interactive(children=(IntSlider(value=2, description='Layers:', layout=Layout(width='400px'), min=1, style=Sli…

This is the Mandelbrot set!   

All from the **simplest** iterative map we could come up with.  

1. The input is always $x=0$
1. Scalar weight $w$ (in fact the weight $w$ is fixed at 1.0)
1. Scalar bias $b$ (both the real and complex case are interesting)
1. The weight and bias are **shared** across all layers

It would appear that weight sharing does not lead to a neural network which is overly simple :-)

https://en.wikipedia.org/wiki/Mandelbrot_set


![](https://upload.wikimedia.org/wikipedia/commons/a/a4/Mandelbrot_sequence_new.gif)

![](https://www.azquotes.com/picture-quotes/quote-the-mandelbrot-set-is-the-most-complex-mathematical-object-known-to-mankind-benoit-mandelbrot-73-97-72.jpg)

### Modern Visualization Best Practices

The notebook above demonstrates several modern practices for interactive scientific visualization:

1. **Plotly over Matplotlib**: Plotly provides native interactivity, better performance for complex plots, and superior widget integration
2. **Responsive Design**: Plots automatically adjust to screen size and provide zoom/pan capabilities
3. **Type-Safe Widgets**: Using properly typed widget parameters with clear descriptions and layouts
4. **Modular Functions**: Each visualization is encapsulated in its own function for reusability
5. **Performance Optimization**: Higher resolution for static plots, efficient computation for interactive elements
6. **Mathematical Precision**: Proper handling of complex numbers and numerical stability

These practices make the notebook more suitable for research presentations and educational use.